In [6]:
import itertools
import sys
from heapq import *
import json
import numpy as np
import pandas as pd
from collections import deque

In [7]:
# 读取weakly connected component.json
with open('../../../技术储备/Tarjan/weakly connected component.json', "r", encoding='utf-8') as r:
    wcc = json.loads(r.read())
wcc=np.array(wcc)
wcc.squeeze()
wcc.shape
biggest_wcc=wcc[0]
biggest_wcc_com=biggest_wcc['component']
# biggest_wcc_com


In [8]:
print(len(biggest_wcc_com))
print("8327" in biggest_wcc_com)

3151
True


In [9]:
v1=json.load(open('../../mc1_vis_system/public/MC1.json'))
# merged_info=pd.read_csv('../V1/merged_info.csv')
nodes_df=pd.DataFrame(v1['nodes'])
links_df=pd.DataFrame(v1['links'])

In [10]:
big_wcc_nodes=nodes_df[nodes_df['id'].isin(biggest_wcc_com)]
big_wcc_links=links_df[(links_df['source'].isin(biggest_wcc_com)) & (links_df['target'].isin(biggest_wcc_com))]
big_wcc_links.info()

<class 'pandas.core.frame.DataFrame'>
Index: 9793 entries, 0 to 10746
Data columns (total 4 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   source  9793 non-null   object 
 1   target  9793 non-null   object 
 2   type    9793 non-null   object 
 3   weight  9792 non-null   float64
dtypes: float64(1), object(3)
memory usage: 382.5+ KB


In [11]:
class Vertex:
    def __init__(self, name):
        self.key = name
        self.neighbor = []
        self.predecessor = None
        self.pre_relationship = None

    def add_neighbor(self, vertex_name):
        self.neighbor.append(vertex_name)

    def add_pre(self, pre, relationship):
        self.predecessor = pre
        self.pre_relationship = relationship


In [12]:
def get_path(dst):
    result_path = []
    while dst.predecessor is not None:
        result_path.append((dst.key,dst.pre_relationship.values[0]))
        dst = dst.predecessor
        # print(dst.key)
    # result_path.append(dst.key)
    return result_path

def clear_pre(all_ver):
    for ver in all_ver:
        all_ver[ver].predecessor = None


In [13]:

# def dijkstra(source, destination, all_vertices):
    # path = {}
    # counter = itertools.count()
    # count_or = next(counter)
    # heap = [(0, count_or, source)]
    # heapify(heap)
    # source.cost = 0
    # visit = []

    # while heap is not None:
    #     # get the top vertex of the heap update its cost to neighbors
    #     tmp_distance, tmp_order, tmp_vertex = heappop(heap)

    #     if tmp_vertex == destination:
    #         break

    #     visit.append(tmp_vertex.key)
    #     for neighbors in tmp_vertex.neighbor:
    #         # neighbors:(name,key)
    #         neighbor_name,neighbor_key=neighbors
    #         if neighbor_name not in visit:
    #             tmp_cost = tmp_distance + tmp_vertex.neighbor[neighbors]
    #             if tmp_cost < all_vertices[neighbor_name].cost:
    #                 relationship=big_wcc_links[(big_wcc_links['source']==tmp_vertex.key) & 
                                            #    (big_wcc_links['target']==neighbor_name)&
                                            #    (big_wcc_links['key']==neighbor_key)]['type']
                    # print(relationship)
                    # all_vertices[neighbor_name].add_pre(tmp_vertex,relationship,neighbor_key)
                    # all_vertices[neighbor_name].cost = tmp_cost
                    # heappush(heap, (int(all_vertices[neighbor_name].cost), next(counter), all_vertices[neighbor_name]))

def bfs(source,destination,all_vertices):
    visited = [source]
    queue = deque([source])
        
    while queue:
        current_vertex = queue.popleft()
        if current_vertex == destination:
            return visited
        for neighbor_key in current_vertex.neighbor:
            # 放的是整个vertex，不是key
            # print(type(neighbor_key))
            neighbor_vertex = all_vertices[neighbor_key]
            if neighbor_vertex not in visited:
                neighbor_vertex = all_vertices[neighbor_key]
                # print(neighbor_vertex.key,neighbor_vertex)
                neighbor_vertex.predecessor = current_vertex
                visited.append(neighbor_vertex)
                queue.append(neighbor_vertex)
                relationship=big_wcc_links[(big_wcc_links['source']==current_vertex.key) & 
                                               (big_wcc_links['target']==neighbor_vertex.key)]['type']
                neighbor_vertex.add_pre(current_vertex,relationship)


In [14]:
all_vertices = {}

for nodes in big_wcc_nodes['id']:
    all_vertices[nodes] = Vertex(nodes)
    
for links in big_wcc_links.iterrows():
    # print(links[1]['source'],links[1]['weight'],links[1]['target'])
    # print(links[1])
    all_vertices[links[1]['source']].add_neighbor(links[1]['target'])
    


In [15]:
for i in big_wcc_nodes['id']:
    if i=="8327":
        print(1)

1


In [16]:
suspicious=["8327","979893388","Mar de la Vida OJSC","Oceanfront Oasis Inc Carriers"]
scc_nodes=['dup77', 'water', 'Gulf', 'salmon', 'Plastic', '19', 'dup626', 
           '629', 'ocean', 'bottom', 'USS', 'PC', 'Oliver', 'HMS Montrose', 
           'Talleyâ\x80\x99s', 'Ships', 'trucking', 'Deepwater Horizon', 'Vessels']

In [17]:
path_common_nodes={}
all_path=[]
with open('./bfs_scc_connection_to_sus2.txt','w') as f:
    for i in  scc_nodes:
        for j in suspicious:
            all_path.append((i,j))
            clear_pre(all_vertices)
            # print(i,j)
            bfs(all_vertices[i],
                     all_vertices[j],all_vertices)
            res=get_path(all_vertices[j])
            f.write(i+"->"+j+'\n')
            f.write(i+",")
            all_path.append(res)
            for k in range(len(res)):
                f.write(res[-k-1][1]+','+res[-k-1][0]+'\n')
                if k<len(res)-1:
                    if res[-k-1][0] in path_common_nodes:
                        path_common_nodes[res[-k-1][0]]+=1
                    else:
                        path_common_nodes[res[-k-1][0]]=1
                # print(res[-k-1])
            # print("----------------------")
            f.write("----------------------\n")

In [104]:
all_path_reverse=[]
sus3=["8327","979893388","Mar de la Vida OJSC"]
with open('./bfs_sus2ves.txt','w') as f:
    for i in  scc_nodes:
        for j in sus3:
            all_path_reverse.append((j,i))
            clear_pre(all_vertices)
            # print(i,j)
            bfs(all_vertices[j],
                     all_vertices[i],all_vertices)
            res=get_path(all_vertices[i])
            f.write(j+"->"+i+'\n')
            f.write(j+",")
            all_path_reverse.append(res)
            for k in range(len(res)):
                f.write(res[-k-1][1]+','+res[-k-1][0]+'\n')
                # print(res[-k-1])
            # print("----------------------")
            f.write("----------------------\n")

In [18]:
len(all_path),all_path

(152,
 [('dup77', '8327'),
  [('8327', 'membership'),
   ('160', 'partnership'),
   ('Erica Hamilton', 'membership'),
   ('464454263', 'ownership'),
   ("Fisherman'sFriend Charter Boat S.A. de C.V. Freight ", 'partnership'),
   ('dup38', 'membership')],
  ('dup77', '979893388'),
  [('979893388', 'ownership'),
   ('Erica Hamilton', 'membership'),
   ('464454263', 'ownership'),
   ("Fisherman'sFriend Charter Boat S.A. de C.V. Freight ", 'partnership'),
   ('dup38', 'membership')],
  ('dup77', 'Mar de la Vida OJSC'),
  [('Mar de la Vida OJSC', 'membership'),
   ('Ancla Azul Company Solutions', 'family_relationship'),
   ('Faroe Islands Shrimp Shark', 'family_relationship'),
   ('979893388', 'ownership'),
   ('Erica Hamilton', 'membership'),
   ('464454263', 'ownership'),
   ("Fisherman'sFriend Charter Boat S.A. de C.V. Freight ", 'partnership'),
   ('dup38', 'membership')],
  ('dup77', 'Oceanfront Oasis Inc Carriers'),
  [('Oceanfront Oasis Inc Carriers', 'ownership'),
   ('979893388', 'o

In [19]:
# 对path_common_nodes按照value排序
path_common_nodes=sorted(path_common_nodes.items(),key=lambda x:x[1],reverse=True)
len(path_common_nodes)

50

In [20]:
clear_pre(all_vertices)
v=bfs(all_vertices["dup77"],all_vertices["8327"],all_vertices)
# for ver in v:
#     print(ver.key)
res=get_path(all_vertices["8327"])
print("dup77")
for i in range(len(res)):
    print(res[-i-1][1],res[-i-1][0])
# print()
# res


dup77
membership dup38
partnership Fisherman'sFriend Charter Boat S.A. de C.V. Freight 
ownership 464454263
membership Erica Hamilton
partnership 160
membership 8327


In [21]:
clear_pre(all_vertices)
bfs(all_vertices["979893388"],all_vertices["Oceanfront Oasis Inc Carriers"],all_vertices)
res=get_path(all_vertices["Oceanfront Oasis Inc Carriers"])
for i in range(len(res)):
    print(res[-i-1])

('Oceanfront Oasis Inc Carriers', 'ownership')


### 画图

In [85]:
# nodes_each_path={}
# nodes_each_path["8327"]
scc_nodes=['dup77', 'water', 'Gulf', 'salmon', 'Plastic', '19', 'dup626', 
           '629', 'ocean', 'bottom', 'USS', 'PC', 'Oliver', 'HMS Montrose', 
           'Talleyâ\x80\x99s', 'Ships', 'trucking', 'Deepwater Horizon', 'Vessels']

In [86]:
suspicious=["8327","979893388","Mar de la Vida OJSC","Oceanfront Oasis Inc Carriers"]
nodes_each_path={i:[] for i in suspicious}
edges={i:{} for i in suspicious}
# for i in suspicious:
#     nodes_each_path[i]=scc_nodes
#     print(len(scc_nodes))
# print(len(nodes_each_path["8327"]))
color_rel={"ownership":'r',"membership":'b',"partnership":'g',"family_relationship":'y'}
key_rel={"ownership":0,"membership":1,"partnership":2,"family_relationship":3}
# nodes

In [87]:
(nodes_each_path["8327"])

[]

In [88]:
node_color={}
for j in scc_nodes:
    node_color[j]="g"
for j in suspicious:
    node_color[j]="r"
# node_colorxs

In [89]:
# edge: (src, tgt, weight)
for path in range(len(all_path)):
    if path % 2 == 0:
        src, tgt = all_path[path]
        rel_pairs = all_path[path+1]
        temp_pre = src
        # print(src,tgt)
        if src not in nodes_each_path[tgt]:
            nodes_each_path[tgt].append(src)
        for pair in range(len(rel_pairs)):
            to_node, rel = rel_pairs[-pair-1]
            # A rel B,这里是B, rel
            if to_node not in node_color:
                node_color[to_node] = "b"
            # 把到达这个终点的路径上的点加入nodes
            if to_node not in nodes_each_path[tgt]:
                nodes_each_path[tgt].append(to_node)
            if (temp_pre, rel, to_node) not in edges[tgt]:
                edges[tgt][(temp_pre, rel, to_node)] = 1
            else:
                edges[tgt][(temp_pre, rel, to_node)] += 1
            temp_pre = to_node

# nodes_each_path


In [90]:
len(nodes_each_path["Mar de la Vida OJSC"])

53

In [91]:
import networkx as nx
import matplotlib.pyplot as plt
%matplotlib inline

In [92]:
four_graphs=[]
for _ in suspicious:
    G = nx.MultiDiGraph()
    G.add_nodes_from(nodes_each_path[_])
    for edge in edges[_]:
        src,rel,tgt=edge
        G.add_edge(src,tgt,weight=edges[_][edge],key=key_rel[rel],color=color_rel[rel])
    four_graphs.append(G)
four_graphs[0]

In [93]:
print(four_graphs[0].edges(data=True))
print("Oceanfront Oasis Inc Carriers" in four_graphs[0].nodes())

[('dup77', 'dup38', {'weight': 1, 'color': 'b'}), ('dup38', "Fisherman'sFriend Charter Boat S.A. de C.V. Freight ", {'weight': 1, 'color': 'g'}), ("Fisherman'sFriend Charter Boat S.A. de C.V. Freight ", '464454263', {'weight': 1, 'color': 'r'}), ('464454263', 'Erica Hamilton', {'weight': 1, 'color': 'b'}), ('Erica Hamilton', '160', {'weight': 1, 'color': 'g'}), ('160', '8327', {'weight': 12, 'color': 'b'}), ('water', '7691', {'weight': 1, 'color': 'b'}), ('7691', 'Spanish Shrimp  Carriers', {'weight': 1, 'color': 'y'}), ('Spanish Shrimp  Carriers', '160', {'weight': 4, 'color': 'r'}), ('Gulf', 'Ocean Fisheries Llc', {'weight': 2, 'color': 'r'}), ('Ocean Fisheries Llc', '160', {'weight': 4, 'color': 'r'}), ('salmon', 'George Ramirez', {'weight': 1, 'color': 'r'}), ('George Ramirez', 'd5b94ebc-e03a-438e-872b-a15db349a748', {'weight': 1, 'color': 'r'}), ('d5b94ebc-e03a-438e-872b-a15db349a748', '903311212', {'weight': 1, 'color': 'b'}), ('903311212', '160', {'weight': 3, 'color': 'r'}), ('

In [94]:
labels = {node: node for node in G.nodes()} 
# labels

In [95]:
import matplotlib as mpl
mpl.use('pgf')
cnt=0
for graph in four_graphs:
    # 绘制节点和边
    plt.figure(figsize=(30, 30))
    pos = nx.kamada_kawai_layout(graph)
    labels = {node: node for node in graph.nodes()}  # 创建节点标签
    nx.draw_networkx_labels(graph, pos, labels, font_size=8, font_color='black', font_family='Arial')  
    nx.draw_networkx_nodes(graph, pos,node_color=[node_color[node] for node in graph.nodes()],node_size=8)
    # nx.draw_networkx_edges(graph, pos)
    nx.draw_networkx_edges(graph, pos, edge_color=[d['color'] for u, v,  d in graph.edges( data=True)],
                                 width=[d['weight'] for u, v, d in graph.edges( data=True)])

    # 显示图形
    plt.axis('off')
    plt.show()
    # print()
    # print('./bfs_paths/to_'+str(_)+'pdf')
    plt.savefig('../V1/bfs_paths/to_'+suspicious[cnt]+'.pdf')
    cnt+=1

/var/folders/2q/mfflg7jd005564r5d6qrvy_h0000gn/T/ipykernel_32149/3974849180.py:17: UserWarning: Matplotlib is currently using pgf, which is a non-GUI backend, so cannot show the figure.
  plt.show()
/var/folders/2q/mfflg7jd005564r5d6qrvy_h0000gn/T/ipykernel_32149/3974849180.py:17: UserWarning: Matplotlib is currently using pgf, which is a non-GUI backend, so cannot show the figure.
  plt.show()
/var/folders/2q/mfflg7jd005564r5d6qrvy_h0000gn/T/ipykernel_32149/3974849180.py:17: UserWarning: Matplotlib is currently using pgf, which is a non-GUI backend, so cannot show the figure.
  plt.show()
/var/folders/2q/mfflg7jd005564r5d6qrvy_h0000gn/T/ipykernel_32149/3974849180.py:17: UserWarning: Matplotlib is currently using pgf, which is a non-GUI backend, so cannot show the figure.
  plt.show()


In [96]:
all_nodes=[]
all_edges={}
for i in nodes_each_path:
    all_nodes+=nodes_each_path[i]
all_nodes=list(set(all_nodes))
len(all_nodes)
for i in edges:
    for j in edges[i]:
            # print(j)
        if j not in all_edges:
            all_edges[j]=1
        else:
            all_edges[j]+=1
len(all_edges)

103

In [97]:
tot_G=nx.MultiDiGraph()
tot_G.add_nodes_from(all_nodes)
for edge in all_edges:
    src,rel,tgt=edge
    tot_G.add_edge(src,tgt,weight=all_edges[edge],key=key_rel[rel],color=color_rel[rel])

plt.figure(figsize=(30, 30))
pos = nx.spring_layout(tot_G)
labels = {node: node for node in tot_G.nodes()}  # 创建节点标签
nx.draw_networkx_labels(tot_G, pos, labels, font_size=8, font_color='black', font_family='Arial')  
nx.draw_networkx_nodes(tot_G, pos,node_color=[node_color[node] for node in tot_G.nodes()],node_size=8)
# nx.draw_networkx_edges(graph, pos)
nx.draw_networkx_edges(tot_G, pos, edge_color=[d['color'] for u, v,  d in tot_G.edges( data=True)],
                                width=[d['weight'] for u, v, d in tot_G.edges( data=True)])

# 显示图形
plt.axis('off')
plt.show()
# print()
# print('./bfs_paths/to_'+str(_)+'pdf')
plt.savefig('../V1/bfs_paths/spring_all.pdf')


/var/folders/2q/mfflg7jd005564r5d6qrvy_h0000gn/T/ipykernel_32149/2883487562.py:18: UserWarning: Matplotlib is currently using pgf, which is a non-GUI backend, so cannot show the figure.
  plt.show()


### 画反向图

In [105]:
sus3=["8327","979893388","Mar de la Vida OJSC"]
nodes_rev={i:[] for i in sus3}
edges_rev={i:{} for i in sus3}
# edge: (src, tgt, weight)
node_color_rev={}
for j in scc_nodes:
    node_color_rev[j]="g"
for j in suspicious:
    node_color_rev[j]="r"
    
for path in range(len(all_path_reverse)):
    if path % 2 == 0:
        src, tgt = all_path_reverse[path]
        rel_pairs = all_path_reverse[path+1]
        temp_pre = src
        # print(src,tgt)
        if tgt not in nodes_rev[src]:
            nodes_rev[src].append(tgt)
        for pair in range(len(rel_pairs)):
            to_node, rel = rel_pairs[-pair-1]
            # A rel B,这里是B, rel
            if to_node not in node_color_rev:
                node_color_rev[to_node] = "b"
            # 把从这个起点出发的路径上的点加入nodes
            if to_node not in nodes_rev[src]:
                nodes_rev[src].append(to_node)
            if (temp_pre, rel, to_node) not in edges_rev[src]:
                edges_rev[src][(temp_pre, rel, to_node)] = 1
            else:
                edges_rev[src][(temp_pre, rel, to_node)] += 1
            temp_pre = to_node

# nodes_each_path


In [111]:
four_graphs_rev=[]
for _ in sus3:
    print(_)
    G = nx.MultiDiGraph()
    G.add_nodes_from(nodes_rev[_])
    for edge in edges_rev[_]:
        src,rel,tgt=edge
        G.add_edge(src,tgt,weight=edges_rev[_][edge],key=key_rel[rel],color=color_rel[rel])
    four_graphs_rev.append(G)
    # print(G)
four_graphs_rev

8327
979893388
Mar de la Vida OJSC


In [113]:
import matplotlib as mpl
mpl.use('pgf')
cnt=0
for graph in four_graphs_rev:
    # 绘制节点和边
    plt.figure(figsize=(30, 30))
    pos = nx.kamada_kawai_layout(graph)
    labels = {node: node for node in graph.nodes()}  # 创建节点标签
    nx.draw_networkx_labels(graph, pos, labels, font_size=8, font_color='black', font_family='Arial')  
    nx.draw_networkx_nodes(graph, pos,node_color=[node_color_rev[node] for node in graph.nodes()],node_size=8)
    # nx.draw_networkx_edges(graph, pos)
    nx.draw_networkx_edges(graph, pos, edge_color=[d['color'] for u, v,  d in graph.edges( data=True)],
                                 width=[d['weight'] for u, v, d in graph.edges( data=True)])

    # 显示图形
    plt.axis('off')
    plt.show()
    # print()
    print('../V1/bfs_paths/from_'+sus3[cnt]+'.pdf')
    plt.savefig('../V1/bfs_paths/from_'+sus3[cnt]+'.pdf')
    cnt+=1

/var/folders/2q/mfflg7jd005564r5d6qrvy_h0000gn/T/ipykernel_32149/3076754423.py:17: UserWarning: Matplotlib is currently using pgf, which is a non-GUI backend, so cannot show the figure.
  plt.show()


../V1/bfs_paths/from_8327.pdf


/var/folders/2q/mfflg7jd005564r5d6qrvy_h0000gn/T/ipykernel_32149/3076754423.py:17: UserWarning: Matplotlib is currently using pgf, which is a non-GUI backend, so cannot show the figure.
  plt.show()


../V1/bfs_paths/from_979893388.pdf


/var/folders/2q/mfflg7jd005564r5d6qrvy_h0000gn/T/ipykernel_32149/3076754423.py:17: UserWarning: Matplotlib is currently using pgf, which is a non-GUI backend, so cannot show the figure.
  plt.show()


../V1/bfs_paths/from_Mar de la Vida OJSC.pdf
